In [1]:
import lab1
from lab1 import token_lower_nostop_stem_list,load_stopword
import xml.etree.ElementTree as ET
from stemming.porter2 import stem

In [2]:
def to_standard_xml(xml_file_path):
    new_file_name = 'standard_' + xml_file_path.split('/')[-1]
    new_file_path = data_folder + new_file_name

    with open(new_file_path,'w',encoding='utf-8') as new_file:
        with open(xml_file_path,'r') as original_file:
            new_file.write('<?xml version="1.0"?>\n')
            new_file.write('<sample>\n')
            for line in original_file:
                if(len(line.strip()) == 0):
                    continue
                new_file.write(line)
            new_file.write('</sample>')
    return new_file_path

def xml_sample(xml_file_path):
    tree = ET.parse(trec_samle_path)
    root = tree.getroot()
    return root

def merge_text(text1,text2):
    merged_text = text1 + text2
    return merged_text

def xml_all_text(xml_file_path,stopword_list,is_sample):
    if (is_sample):
        # sample
        standard_xml_path = to_standard_xml(sample_xml_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text,token_lower_nostop_stem_list(child.find('Text').text, stopword_list)) for child in
                         root.findall("./DOC")]
    else:
        # trec
        standard_xml_path = to_standard_xml(trec_samle_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text, token_lower_nostop_stem_list(merge_text(child.find('TEXT').text,child.find('HEADLINE').text), stopword_list)) for child in root.findall("./DOC")]

    return all_text_list

In [3]:
is_sample = 0
data_folder = '../data/collections/'
sample_txt_path = '../data/collections/sample.txt'
trec_samle_path = '../data/collections/trec.sample.xml'
sample_xml_path = '../data/collections/sample.xml'
stopword_filepath = '../data/englishST.txt'

In [4]:
#load stop words
stopword_list = load_stopword(stopword_filepath)

将非标准xml文件转化成标准xml文件, 经过去除停用词, 化为stem

all_text_list= [doc1_stemlist, doc2_stemlist, .......] 

In [5]:
#for xml file
if(is_sample):
    all_text_list = xml_all_text(sample_xml_path,stopword_list,is_sample)
else:
    all_text_list = xml_all_text(trec_samle_path,stopword_list,is_sample)

In [6]:
def inverted_index(all_text_list):
    '''
    return: # index dic[stem]={'doc_id':position}
    '''

    #get stem
    stem_set = set()
    for current_doc_tupple in all_text_list:
        for current_stem in current_doc_tupple[1]:
            stem_set.add(current_stem)

    index_dic = {}
    for current_stem in stem_set:
        current_stem_doc_position_dic = {}
        for current_doc_tupple in all_text_list:
            current_doc_id = current_doc_tupple[0]
            current_doc_text_list = current_doc_tupple[1]

            if (current_stem not in current_doc_text_list):
                continue

            position_list = [index+1 for index in range(len(current_doc_text_list))
                             if current_doc_text_list[index] == current_stem]
            current_stem_doc_position_dic[str(current_doc_id)] = position_list
        index_dic[current_stem] = current_stem_doc_position_dic
    return index_dic

In [7]:
index_dic = inverted_index(all_text_list)
index_dic

{'huangshi': {'3434': [196, 302]},
 '138': {'128': [86, 106],
  '137': [539],
  '3567': [231],
  '3774': [87, 107],
  '3786': [650]},
 'substanti': {'12': [130],
  '17': [55],
  '19': [16],
  '33': [38],
  '49': [189],
  '92': [53],
  '130': [6],
  '141': [688],
  '154': [58],
  '165': [92],
  '205': [23],
  '226': [62],
  '251': [202],
  '310': [60],
  '323': [199],
  '336': [298],
  '354': [62],
  '366': [130],
  '370': [449, 465],
  '3329': [62, 145, 522],
  '3333': [7],
  '3337': [92],
  '3345': [74],
  '3350': [8],
  '3396': [249],
  '3404': [61],
  '3405': [203],
  '3442': [195],
  '3444': [113],
  '3463': [108, 232, 238],
  '3495': [246],
  '3509': [70],
  '3563': [138],
  '3604': [141],
  '3654': [94],
  '3693': [384],
  '3719': [160],
  '3721': [41],
  '3726': [423, 887],
  '3741': [152],
  '3786': [496],
  '3788': [240],
  '3791': [91],
  '3810': [90],
  '3811': [33],
  '3815': [30, 66],
  '3819': [25, 92],
  '3826': [132],
  '3829': [49],
  '3843': [103, 111],
  '3927': [524

In [8]:
#output sample
if(is_sample):
    output_path = '../data/collections/sample.index'
else:
    output_path = '../data/collections/trec.index'

In [22]:
def output_index(index_dic,file_path):
    sorted_index_dic_list = sorted(index_dic.items(),key=lambda x:x[0],reverse=False)
    with open(file_path,'w',encoding='utf-8') as file_1:
        for current_tupple in sorted_index_dic_list:
            current_stem = current_tupple[0]
            current_stem_doc_position_dic = current_tupple[1]
            file_1.write(current_stem)
            file_1.write(':\n')
            for current_doc_id,current_doc_id_position_list in current_stem_doc_position_dic.items():
                file_1.write('\t')
                file_1.write(str(current_doc_id))
                file_1.write(': ')
                current_doc_id_position_list_new = map(lambda x: str(x), current_doc_id_position_list)
                # print(','.join(current_doc_id_position_list_new))
                file_1.write(','.join(current_doc_id_position_list_new))
                file_1.write('\n')
            file_1.write('\n')

    return

In [23]:
output_index(index_dic,output_path)

In [24]:
#output sample
if(is_sample):
    index_path = '../data/collections/sample.index'
else:
    index_path = '../data/collections/trec.index'

In [25]:
def load_inverted_index(index_path):
    result_inverted_index = {}
    with open(index_path,'r') as f1:
        for line in f1:
            line = line.strip()
            if(line.endswith(':')):
                current_stem = line.replace(':','')
                result_inverted_index[current_stem] = {}
                continue
                
            if(len(line)==0):
                continue
                
            temp_split_list = line.split(': ')
            current_doc_id,str_position_list = temp_split_list[0],temp_split_list[1]
            current_position_list = [int(current_position) for current_position in str_position_list.split(',')]
            result_inverted_index[current_stem][current_doc_id] = current_position_list
    return result_inverted_index

In [26]:
loaded_index_dic = load_inverted_index(index_path)

In [27]:
loaded_index_dic

{'0': {'14': [54, 112, 122, 130],
  '15': [11, 22, 25, 276],
  '21': [166, 172],
  '23': [153],
  '37': [12, 44, 108, 121],
  '59': [98,
   115,
   136,
   157,
   189,
   248,
   401,
   444,
   465,
   496,
   520,
   549,
   568,
   598,
   634],
  '118': [124],
  '128': [175],
  '135': [180, 187],
  '137': [559, 561],
  '144': [166],
  '145': [228, 329, 361],
  '146': [11,
   193,
   196,
   209,
   211,
   213,
   235,
   248,
   287,
   310,
   326,
   339,
   352,
   365,
   378,
   380,
   391,
   393,
   395,
   418,
   420,
   431,
   433,
   444,
   446,
   470,
   483,
   485,
   487,
   525,
   537,
   539],
  '216': [33],
  '217': [20, 99],
  '222': [67],
  '268': [179, 184, 189, 190, 213, 217, 234, 246, 286, 291],
  '275': [215, 261, 281, 295],
  '287': [455,
   506,
   511,
   513,
   518,
   531,
   532,
   541,
   545,
   546,
   560,
   574,
   611,
   635,
   649,
   651,
   654,
   665,
   678,
   679,
   690,
   692,
   698,
   703,
   705,
   735,
   747,
   755,

In [28]:
index_dic

{'huangshi': {'3434': [196, 302]},
 '138': {'128': [86, 106],
  '137': [539],
  '3567': [231],
  '3774': [87, 107],
  '3786': [650]},
 'substanti': {'12': [130],
  '17': [55],
  '19': [16],
  '33': [38],
  '49': [189],
  '92': [53],
  '130': [6],
  '141': [688],
  '154': [58],
  '165': [92],
  '205': [23],
  '226': [62],
  '251': [202],
  '310': [60],
  '323': [199],
  '336': [298],
  '354': [62],
  '366': [130],
  '370': [449, 465],
  '3329': [62, 145, 522],
  '3333': [7],
  '3337': [92],
  '3345': [74],
  '3350': [8],
  '3396': [249],
  '3404': [61],
  '3405': [203],
  '3442': [195],
  '3444': [113],
  '3463': [108, 232, 238],
  '3495': [246],
  '3509': [70],
  '3563': [138],
  '3604': [141],
  '3654': [94],
  '3693': [384],
  '3719': [160],
  '3721': [41],
  '3726': [423, 887],
  '3741': [152],
  '3786': [496],
  '3788': [240],
  '3791': [91],
  '3810': [90],
  '3811': [33],
  '3815': [30, 66],
  '3819': [25, 92],
  '3826': [132],
  '3829': [49],
  '3843': [103, 111],
  '3927': [524

## query

In [29]:
query_dic = {'q1': 'Scotland','q2': 'Window','q3': 'replacing','q4': 'condemning','q5': 'income OR taxes','q6': 'income AND NOT taxes','q7': '"income taxes"','q8': '#10(income, taxes)','q9': '"middle east" AND peace'}
query_dic

{'q1': 'Scotland',
 'q2': 'Window',
 'q3': 'replacing',
 'q4': 'condemning',
 'q5': 'income OR taxes',
 'q6': 'income AND NOT taxes',
 'q7': '"income taxes"',
 'q8': '#10(income, taxes)',
 'q9': '"middle east" AND peace'}

In [86]:
def get_doc_id_set(current_inverted_dic):
    doc_id_set = set()
    for current_stem,current_doc_position_dic in current_inverted_dic.items():
        for current_doc_id in current_doc_position_dic.keys():
            doc_id_set.add(str(current_doc_id))
    return doc_id_set

def query_word(current_inverted_dic,current_word,is_not=0):
    current_word_stem = stem(current_word.lower())
    if(is_not):
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                doc_id_set = get_doc_id_set(current_inverted_dic)
                stem_doc_list = set(current_index_stem_position.keys())
                return list(doc_id_set.difference(stem_doc_list))
    else:
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                return list(current_index_stem_position.keys())
    raise RuntimeError('Query not found.')
    return

# OR
def union_list(a,b):
    result_list = list(set(a).union(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

# AND
def intersection_list(a,b):
    result_list = list(set(a).intersection(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

def probability_query(doc_word_pos1,doc_word_pos2,current_distance):
    result_list = []
    for current_docid_1,current_positionlist_1 in doc_word_pos1.items():
        if(current_docid_1 not in doc_word_pos2.keys()):
            continue
        current_positionlist_2 = doc_word_pos2[current_docid_1]
        i = 0
        j = 0

        while((i <= len(current_positionlist_1) - 1) and (j <= len(current_positionlist_2) - 1)):
            if(int(current_positionlist_1[i]) > int(current_positionlist_2[j]) + current_distance):
                j+=1
                continue
            elif(int(current_positionlist_1[i]) < int(current_positionlist_2[j]) - current_distance):
                i+=1
                continue
            else:
                result_list.append(current_docid_1)
                break
    return result_list

#phrase
def phrase_query(current_inverted_dic,query_phrase):
    phrase_list = [ stem(current_word.strip()) for current_word in query_phrase.replace('"','').split()]
    result_list = probability_query(current_inverted_dic[phrase_list[0]],current_inverted_dic[phrase_list[1]],1)
    return result_list

# word position querry
def word_position_querry(current_inverted_dic, current_word):
    '''
    return: {'doc_id':[position_list]}
    '''
    if(current_word in current_inverted_dic.keys()):
        return current_inverted_dic[current_word]
    
    raise RuntimeError('Query not found.')
    return




In [88]:
for current_query_id, current_query in query_dic.items():
    if(current_query_id == 'q1'):
        current_result = query_word(loaded_index_dic,current_query)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
        
        
    elif(current_query_id == 'q2'):
        current_result = query_word(loaded_index_dic,current_query)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)

    
    elif(current_query_id == 'q3'):
        current_result = query_word(loaded_index_dic,current_query)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
        
    
    elif(current_query_id == 'q4'):
        current_result = query_word(loaded_index_dic,current_query)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
    
    elif(current_query_id == 'q5'):
        query_word_list = [current_query_word.strip() for current_query_word in current_query.split('OR')]
        current_result = union_list(query_word(loaded_index_dic,query_word_list[0]),query_word(loaded_index_dic,query_word_list[1]))

        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
        
    elif(current_query_id == 'q6'):
        query_word_list = [current_query_word.strip() for current_query_word in current_query.split('AND')]
        current_result = intersection_list(query_word(loaded_index_dic,query_word_list[0]),query_word(loaded_index_dic,query_word_list[1].replace('NOT','').strip(),is_not = 1))

        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
    
    elif(current_query_id == 'q7'):
        current_result = phrase_query(loaded_index_dic,current_query)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
    
    elif(current_query_id == 'q8'):
        current_querry_list = current_query.split('(')
        current_word_distance = int(current_querry_list[0].replace('#','').strip())
        current_querry_stem_list = [stem(temp_word.strip()) for temp_word in current_querry_list[1].replace(')','').split(',')]
        
        current_result = probability_query(loaded_index_dic[current_querry_stem_list[0]],loaded_index_dic[current_querry_stem_list[1]],10)
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
    
    # 'q9'
    else:
        current_querry_list = current_query.split('AND')
        current_result = intersection_list(phrase_query(loaded_index_dic,current_querry_list[0].strip()),query_word(loaded_index_dic,current_querry_list[1].strip()))
        print('%s: %s'%(current_query_id,current_query))
        print('\t',current_result)
    print()

q1: Scotland
	 ['16', '94', '96', '143', '272', '351', '370', '3330', '3334', '3336', '3338', '3376', '3485', '3504', '3505', '3532', '3533', '3561', '3565', '3629', '3654', '3820', '3826', '3932', '3938']

q2: Window
	 ['30', '55', '59', '145', '160', '272', '374', '3463', '3639', '3782', '3788', '3906', '3909', '3910', '3917', '3930']

q3: replacing
	 ['16', '28', '31', '51', '92', '97', '131', '135', '136', '138', '140', '169', '195', '197', '218', '242', '256', '274', '276', '325', '3325', '3342', '3361', '3370', '3387', '3397', '3403', '3410', '3411', '3413', '3418', '3437', '3438', '3461', '3494', '3504', '3521', '3531', '3532', '3533', '3561', '3598', '3623', '3654', '3660', '3662', '3668', '3693', '3700', '3716', '3720', '3724', '3734', '3745', '3788', '3805', '3828', '3867', '3869', '3876', '3886', '3895', '3896', '3907', '3918', '3921', '3930', '3935']

q4: condemning
	 ['113', '309', '354', '3374', '3674', '3678']

q5: income OR taxes
	 ['3', '14', '16', '23', '24', '32', '3

'incom'